# RESEAU DE NEURONES POUR PREDICTION DE TOKEN

Dans ce tutoriel nous utilisons deux sorties pour prédire la probabilité des tokens `0` ou `1`. 

# On crée le modèle

In [1]:
import torch.nn as nn

# Define a neural network with two outputs (representing token probabilities)
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(1, 2)  # One inputs → Two outputs (for two tokens)

    def forward(self, x):
        return self.linear(x)  # No softmax here (included in loss function)

In [2]:
model = Model()

# On définit les paramètres d'apprentissage

In [7459]:
# Learning rate
lr = 0.1

In [7460]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy for classification
optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer


# La fonction fit()

In [7461]:
import torch

def fit(input, target):
    input_tensor = torch.tensor(input, dtype=torch.float)
    target_tensor = torch.tensor(target, dtype=torch.long)
    labels = torch.nn.functional.one_hot(target_tensor, num_classes=2).to(torch.float)
    #labels = torch.argmax(target_tensor, dim=1)  # Convert one-hot to class indices

    optimizer.zero_grad()  # Reset gradients
    outputs = model(input_tensor)  # Forward pass
    loss = criterion(outputs, labels)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    print(f"Loss: {loss.item():.6f}")


# On entraine le modèle

Entrainer jusqu'a avoir une loss < 0.5

In [7462]:
fit([2], 1)
fit([3], 0)

Loss: 0.769202
Loss: 1.099631


# La fonction predict()

In [7463]:
def predict(input):
    input_tensor = torch.tensor(input, dtype=torch.float)
    output_tensor = model(input_tensor)
    print("logits:", output_tensor.detach().numpy())
    probs = torch.nn.functional.softmax(output_tensor, dim=0)  # Convert logits to probabilities
    print("probabilités:", probs.detach().numpy())
    return torch.argmax(output_tensor, dim=0).item()

In [7464]:
predicted_outcome = predict([2])
print("Prediction pour 2:", predicted_outcome)
predicted_outcome = predict([3])
print("Prediction pour 3:", predicted_outcome)

logits: [0.6245817  0.91014135]
probabilités: [0.42909127 0.5709087 ]
Prediction pour 2: 1
logits: [0.87807   1.3371849]
probabilités: [0.3871958 0.6128042]
Prediction pour 3: 1


# TEST MODELE AVEC DEUX ENTREES

In [7465]:
# Define a neural network with two outputs (representing token probabilities)
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(2, 2)  # Two inputs → Two outputs (for two tokens)

    def forward(self, x):
        return self.linear(x)  # No softmax here (included in loss function)

In [7466]:
model = Model()
optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

In [7467]:
fit([2, 2], 0)
fit([2, 3], 1)
fit([3, 2], 1)
fit([3, 3], 0)

Loss: 2.307905
Loss: 0.118523
Loss: 0.612002
Loss: 1.232157


On voit que la loss se stabilise vers 0.7. Le modèle n'apprend pas correctement.

In [7455]:
predicted_outcome = predict([2, 2])
print("Prediction pour 2:", predicted_outcome)
predicted_outcome = predict([2, 3])
print("Prediction pour 3:", predicted_outcome)
predicted_outcome = predict([3, 2])
print("Prediction pour 2:", predicted_outcome)
predicted_outcome = predict([3, 3])
print("Prediction pour 3:", predicted_outcome)

logits: [ 0.00850408 -0.23773761]
probabilités: [0.5612512  0.43874875]
Prediction pour 2: 0
logits: [-0.07946078  0.16243005]
probabilités: [0.4398204  0.56017953]
Prediction pour 3: 1
logits: [-0.44530684  0.19091642]
probabilités: [0.34610078 0.65389925]
Prediction pour 2: 1
logits: [ 0.07588916 -0.33085382]
probabilités: [0.60030663 0.39969334]
Prediction pour 3: 0


Les probabilités sont proches de 0.5 et les prédictions sont erronnées. 

# TEST MODELE AVEC UNE COUCHE CACHEE

Modèle avec une couche cachée.

6 neurones cachés semble un bon compromis.

In [12217]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(2, 6)
        self.fc2 = nn.Linear(6, 2)

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))  # Apply non-linearity
        return self.fc2(x)  # Logits (CrossEntropyLoss handles softmax)


ChatGPT recommande d'utiliser SGD optimizer (Stochasit Gradient Descent, best for online training).

Learning rate de 0.2 semble le mieux.

In [12218]:
model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.2)  # SGD optimizer

On calcule l'accuracy après chaque passe d'entrainement.

In [12219]:
def fit(inputs, targets):
    input_tensor = torch.tensor(inputs, dtype=torch.float)
    # input_tensor = torch.randn_like(input_tensor) * 0.01 (voir si le modèle apprend des tendances)
    target_tensor = torch.tensor(targets, dtype=torch.long)
    labels = torch.nn.functional.one_hot(target_tensor, num_classes=2).to(torch.float)
    #labels = torch.argmax(target_tensor, dim=1)  # Convert one-hot to class indices

    optimizer.zero_grad()  # Reset gradients
    outputs = model(input_tensor)  # Forward pass
    loss = criterion(outputs, labels)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Check accuracy (we expect 100% accuracy)
    predictions = torch.argmax(outputs, dim=1)
    accuracy = (predictions == target_tensor).float().mean().item()

    print(f"Loss: {loss.item():.6f}, Accuracy: {accuracy * 100:.0f}%")


Entrainer le modèle jusqu'a obtenir une accuracy de 100%

In [12220]:
iteration = 1

In [12441]:
print("Iteration:", iteration)
fit([[2, 2], [2, 3], [3, 2],[3, 3]], [0, 1, 1, 0])
iteration += 1

Iteration: 221
Loss: 0.050310, Accuracy: 100%


On atteint une accuracy de 100% la plupart des fois en moins de 200 iterations avec 6 neurones cachés. 
Il reste cependant des fois ou il ne converge pas.

4 neurones semblent insuffisant, 8 semblent trop.

On atteint une accuracy de 100% la plupart des fois en moins de 200 iterations avec 6 neurones cachés. 
Cependant, certaines fois il ne converge pas.

4 neurones semblent insuffisant, 8 semblent trop.